# 合并个人代币

基于merge_token的场景，bob有2+2个token，将其合并之

使用了request包

In [7]:
# metadate好像只能跟随事务
# import BigchainDB and create an object
from bigchaindb_driver import BigchainDB
bdb_root_url = 'http://192.168.1.110:9984'
bdb = BigchainDB(bdb_root_url)

alice_private_key='88L2BJC9eNtSWhpPwWqqsLDRGz7aBPhuRNyfsWx4QxWR'
alice_public_key='HWkENox4DM4Tp3qSfYW8igndpog9GpKFzB7Tp7yXgpBq'
bob_private_key='HwLCf9fbhm6BHTagY5aC1uVKR6sz57h7viuS8DUR9x34'
bob_public_key='3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'

```
192.168.1.110:9984/api/v1/outputs?public_key=3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm&spent=false

[{"transaction_id": "0cac4c757e26eca6599a0cce376796a3b088635af6c7d3558917bd2939d91af0", "output_index": 0}, 
{"transaction_id": "c2b0c95c02239532f11f5d8c263d91cddbd6acf9146ac70d1a1171b8b61da050", "output_index": 0}]
```

In [33]:
import requests
import json
# 某人未消耗的输出
payload = {'public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm', 'spent': 'false'}
r = requests.get("http://192.168.1.110:9984/api/v1/outputs", params=payload)
# r.url
# r.text
# print(r.json)  # 不能解码到body
text = json.loads(r.text)
text

[{'transaction_id': '1f83959db7d066da989bfb5902135b807e5f45aa34e6cbd43ceb95289e709338',
  'output_index': 0},
 {'transaction_id': '41551975ddf934c0c845f65a71e9596b7429c0e78ff3d7f9b7979183013da3b3',
  'output_index': 0},
 {'transaction_id': '81a900d97dd3ba69a8746a5ef5a66bb199ff84be5699fc97f8e7b45765265e4b',
  'output_index': 0},
 {'transaction_id': '28405b3c44df8d56305c3951eb6ad368a0719e51fa3e34bcb602c906a3d64e9e',
  'output_index': 0}]

In [19]:
# 某资产的事务
# /api/v1/transactions?asset_id=e0d22c415cd5a398c3373e8f93b065fc65c5cb6ddac5d984315270b4e5fa895a

payload2 = {'asset_id': 'e0d22c415cd5a398c3373e8f93b065fc65c5cb6ddac5d984315270b4e5fa895a'}
r2 = requests.get("http://192.168.1.110:9984/api/v1/transactions", params=payload2)
# text2 = json.loads(r2.text)
text2 = json.loads(r2.text)
text2

[{'inputs': [{'owners_before': ['HWkENox4DM4Tp3qSfYW8igndpog9GpKFzB7Tp7yXgpBq'],
    'fulfills': None,
    'fulfillment': 'pGSAIPVZtx4o_rOegIGfPcYb4EaFDHeiZeOMo33DyRobp6QIgUBTg2yCqp1lOE3Mk0MUyGCFNerv_rNmsBW7IZOq0nRxMVezxKI8WiVfu7BQi1VHVmjK0XkB_lOA12vVtAwmt-0H'}],
  'outputs': [{'public_keys': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
    'condition': {'details': {'type': 'ed25519-sha-256',
      'public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'},
     'uri': 'ni:///sha-256;t1GM7Bud9-p2hvzkPDn8AN8FRSc8azG8u0BG_KLswiE?fpt=ed25519-sha-256&cost=131072'},
    'amount': '10'}],
  'operation': 'CREATE',
  'metadata': None,
  'asset': {'data': {'token_for': {'game_boy2': {'serial_number2': 'LR35902'}},
    'description': '余额2'}},
  'version': '2.0',
  'id': 'e0d22c415cd5a398c3373e8f93b065fc65c5cb6ddac5d984315270b4e5fa895a'},
 {'inputs': [{'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
    'fulfills': {'transaction_id': 'e0d22c415cd5a398c3373e8f93b065fc65c

In [20]:
len(text2)

3

# 如何获取 某人某资产剩余的代币

通过输出，能得到某人未消耗的代币，还需要确定某资产
按照现在的api有2种方案：

1. 遍历所有的未消耗输出，以资产id为过滤条件；坑，第一次的创建事务，记录的不是资产id，而是资产本身；解决方法，主动将第一次的创建消耗掉（给自己）
2. 遍历该资产相关的所有事务，判断是否未消耗；问题，随着事务的增加，体量会迅速增大，匹配算法复杂；不用分析了，该方法丢弃；

In [30]:
# 遍历某人未消耗输出的事务
bob_device = 'e0d22c415cd5a398c3373e8f93b065fc65c5cb6ddac5d984315270b4e5fa895a'
transfer_asset = {
     'id': bob_device,
}
transfer_inputs = []
bob_token_amount = 0
for i in text:
#     print(i)
    # 获取事务记录
    ri = requests.get("http://192.168.1.110:9984/api/v1/transactions/" + i['transaction_id'])
    texti = json.loads(ri.text)
    # filter 过滤某资产相关事务
    try:
        asset_id = texti['asset']['id']
    except KeyError:
        print("Error: 新创建资产没有['id']")
    else:
        if asset_id == bob_device:
            output = texti['outputs'][i['output_index']]
            output_index = i['output_index']
            transaction_id = i['transaction_id']
#             asset_id = bob_device
            bob_token_amount = bob_token_amount + int(output['amount'])
            transfer_input = {
                'fulfillment': output['condition']['details'],
                'fulfills': {
                    'output_index': output_index,
                    'transaction_id': transaction_id,
                },
                'owners_before': output['public_keys'],
            }
            transfer_inputs.append(transfer_input)  # (dict,)加上,才能转换为tuple
                                   
print(transfer_inputs," ",bob_token_amount)


Error: 新创建资产没有['id']
[{'fulfillment': {'type': 'ed25519-sha-256', 'public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'}, 'fulfills': {'output_index': 0, 'transaction_id': '0cac4c757e26eca6599a0cce376796a3b088635af6c7d3558917bd2939d91af0'}, 'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm']}, {'fulfillment': {'type': 'ed25519-sha-256', 'public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'}, 'fulfills': {'output_index': 0, 'transaction_id': 'c2b0c95c02239532f11f5d8c263d91cddbd6acf9146ac70d1a1171b8b61da050'}, 'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm']}]   4


# 如上过滤出bob的代币2+2，现在合并之
事务的prepare方法param input，可以是list，即可以多个输入

# 合并bob的token
```
output_index = 0
output = fulfilled_token_tx['outputs'][output_index]
transfer_input = {
    'fulfillment': output['condition']['details'],
    'fulfills': {
        'output_index': output_index,
        'transaction_id': fulfilled_token_tx['id'],
    },
    'owners_before': output['public_keys'],
}
transfer_asset = {
     'id': fulfilled_token_tx['id'],
}
prepared_transfer_tx = bdb.transactions.prepare(
     operation='TRANSFER',
     asset=transfer_asset,
     inputs=transfer_input,
     recipients=[([bob.public_key], 2), ([alice.public_key], 8)]
)
fulfilled_transfer_tx = bdb.transactions.fulfill(prepared_transfer_tx, private_keys=bob.private_key)
fulfilled_transfer_tx
```
基本上到outputs这一级就够了 // todo

In [31]:
# prepare
prepared_transfer_tx = bdb.transactions.prepare(
     operation='TRANSFER',
     asset=transfer_asset,
     inputs=transfer_inputs,
     recipients=[([bob_public_key], bob_token_amount)]
)
fulfilled_transfer_tx = bdb.transactions.fulfill(prepared_transfer_tx, private_keys=bob_private_key)
fulfilled_transfer_tx

{'inputs': [{'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
   'fulfills': {'transaction_id': '0cac4c757e26eca6599a0cce376796a3b088635af6c7d3558917bd2939d91af0',
    'output_index': 0},
   'fulfillment': 'pGSAICNu3miMPbgnonQOrcbGgUknVoZB1S3vKt5lGBSitv90gUB7DBfk2YATq8ZYj4tCmTwrCNvThdHpe4PVMpWPRwIaUalwGPpAIsrHFeELCJcpr5DI_fSz5-VKtr6LwmYPiRYP'},
  {'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
   'fulfills': {'transaction_id': 'c2b0c95c02239532f11f5d8c263d91cddbd6acf9146ac70d1a1171b8b61da050',
    'output_index': 0},
   'fulfillment': 'pGSAICNu3miMPbgnonQOrcbGgUknVoZB1S3vKt5lGBSitv90gUDZFV2Qd2WLzK6aH9gEzUekW2Ge3IguDVCcMaNEENYhEUe-pQaAs1bFHjl2k15q1_4lei51ClcTQya-6ZfhwAwA'}],
 'outputs': [{'public_keys': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'},
    'uri': 'ni:///sha-256;t1GM7Bud9-p2hvzkPDn8AN8FRSc8azG8u0BG_KLswiE?fpt=ed

In [32]:
sent_transfer_tx = bdb.transactions.send_commit(fulfilled_transfer_tx)
print(sent_transfer_tx == fulfilled_transfer_tx)

True


In [34]:
# 如上合并成功，bob有了完整的4token
# 28405b3c44df8d56305c3951eb6ad368a0719e51fa3e34bcb602c906a3d64e9e
r3 = requests.get("http://192.168.1.110:9984/api/v1/transactions/28405b3c44df8d56305c3951eb6ad368a0719e51fa3e34bcb602c906a3d64e9e")
text3 = json.loads(r3.text)
text3

{'inputs': [{'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
   'fulfills': {'transaction_id': '0cac4c757e26eca6599a0cce376796a3b088635af6c7d3558917bd2939d91af0',
    'output_index': 0},
   'fulfillment': 'pGSAICNu3miMPbgnonQOrcbGgUknVoZB1S3vKt5lGBSitv90gUB7DBfk2YATq8ZYj4tCmTwrCNvThdHpe4PVMpWPRwIaUalwGPpAIsrHFeELCJcpr5DI_fSz5-VKtr6LwmYPiRYP'},
  {'owners_before': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
   'fulfills': {'transaction_id': 'c2b0c95c02239532f11f5d8c263d91cddbd6acf9146ac70d1a1171b8b61da050',
    'output_index': 0},
   'fulfillment': 'pGSAICNu3miMPbgnonQOrcbGgUknVoZB1S3vKt5lGBSitv90gUDZFV2Qd2WLzK6aH9gEzUekW2Ge3IguDVCcMaNEENYhEUe-pQaAs1bFHjl2k15q1_4lei51ClcTQya-6ZfhwAwA'}],
 'outputs': [{'public_keys': ['3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'],
   'condition': {'details': {'type': 'ed25519-sha-256',
     'public_key': '3PKKhLTbaFSjpjdEtNYqPTSrgp17Vur25NwVjQNKK7Hm'},
    'uri': 'ni:///sha-256;t1GM7Bud9-p2hvzkPDn8AN8FRSc8azG8u0BG_KLswiE?fpt=ed

# 2019年4月21日21点27分 done